In [27]:
DROP TABLE IF EXISTS TRANSACTIONINFO;
DROP TABLE IF EXISTS BANKACCOUNTS;


Commands completed successfully.

Total execution time: 00:00:00.006

In [28]:
CREATE TABLE BankAccounts(
    ID INT IDENTITY(1,1) CONSTRAINT PK_BANK_ACCOUNT_ID PRIMARY KEY,
    NAME VARCHAR(100),
    PASSWORD VARCHAR(100),
    AMOUNT FLOAT 
);

CREATE TABLE TransactionInfo(
    ID INT IDENTITY(1,1) CONSTRAINT PK_TRANSACTION_ID PRIMARY KEY,
    ACCOUNT_ID INT CONSTRAINT FK__TRANSACTION_INFO__ACCOUNT_ID__BANK_ACCOUNT_ID FOREIGN KEY REFERENCES BANKACCOUNTS(ID),
    DESCRIPTION VARCHAR(100),
    TRANSACTION_TYPE INT,  --     CREDIT: 1  DEBIT: -1
    AMOUNT  FLOAT
);

Commands completed successfully.

Total execution time: 00:00:00.020

In [29]:
INSERT 
    INTO BANKACCOUNTS(NAME,AMOUNT,PASSWORD)
    VALUES
        ('Vivek', 5000,'P@SS'),
        ('Sanjay',5000,'P@SS');

(2 rows affected)

Total execution time: 00:00:00.015

In [30]:
SELECT * FROM BANKACCOUNTS

(2 rows affected)

Total execution time: 00:00:00.002

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,5000
2,Sanjay,P@SS,5000


In [31]:
DROP PROCEDURE IF EXISTS DEPOSIT;

Commands completed successfully.

Total execution time: 00:00:00.011

In [32]:
CREATE PROCEDURE DEPOSIT
    @ID INT,
    @AMOUNT FLOAT
AS
BEGIN

    DECLARE @BALANCE FLOAT;
    DECLARE @COUNT INT;



    SELECT
        @COUNT= COUNT(*),
        @BALANCE= MAX(AMOUNT)
    FROM
        BANKACCOUNTS
    WHERE
        ID=@ID;

    IF @COUNT=0
    BEGIN
        RAISERROR('Invalid Account Number',15,1)
        RETURN 
    END


    INSERT 
        INTO TRANSACTIONINFO(ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT)
        VALUES (@ID, 'Deposit' , 1, @AMOUNT);



    UPDATE BankAccountS
    SET AMOUNT= @BALANCE + @AMOUNT
    WHERE
    ID=@ID;

    

END


Commands completed successfully.

Total execution time: 00:00:00.016

In [33]:
EXEC DEPOSIT @ID=1, @AMOUNT=1000

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.018

In [35]:
SELECT * FROM BANKACCOUNTS;

SELECT * FROM TRANSACTIONINFO;

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.006

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,6000
2,Sanjay,P@SS,5000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000


In [36]:
CREATE PROCEDURE WITHDRAW
    @ID INT,
    @AMOUNT FLOAT,
    @PASSWORD VARCHAR(100)
AS
BEGIN

    DECLARE @BALANCE FLOAT;
    DECLARE @COUNT INT;
    DECLARE @ACCOUNT_PASSWORD VARCHAR(100);
    



    SELECT
        @COUNT= COUNT(*),
        @BALANCE= MAX(AMOUNT)
    FROM
        BANKACCOUNTS
    WHERE
        ID=@ID;

    IF @COUNT=0
    BEGIN
        RAISERROR('Invalid Account Number',15,1)
        RETURN 
    END

    SELECT       
        @ACCOUNT_PASSWORD=PASSWORD
    FROM
        BANKACCOUNTS

    WHERE ID=@ID

    GROUP BY ID,PASSWORD;


    IF @PASSWORD != @ACCOUNT_PASSWORD
    BEGIN
        RAISERROR('Invalid Password',15,2)
        RETURN
    END

    IF @BALANCE < @AMOUNT
    BEGIN
        RAISERROR('Insufficient Balance',15,3)
        RETURN
    END

    

    INSERT 
        INTO TRANSACTIONINFO(ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT)
        VALUES (@ID, 'Withdraw' , -1, @AMOUNT);



    UPDATE BankAccountS
    SET AMOUNT= @BALANCE - @AMOUNT
    WHERE
    ID=@ID;

    

END


Commands completed successfully.

Total execution time: 00:00:00.007

In [37]:
EXEC WITHDRAW 1, 1000, 'P@SS'

SELECT * FROM BANKACCOUNTS;
SELECT * FROM TRANSACTIONINFO;

(1 row affected)

(1 row affected)

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.021

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,5000
2,Sanjay,P@SS,5000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000
2,1,Withdraw,-1,1000


In [38]:
EXEC WITHDRAW 10, 1000, 'PASS'


: Msg 50000, Level 15, State 1, Procedure WITHDRAW, Line 25
Invalid Account Number

Total execution time: 00:00:00.018

In [39]:
EXEC WITHDRAW 1, 1000, 'WRONG-PASSWORD'

: Msg 50000, Level 15, State 2, Procedure WITHDRAW, Line 41
Invalid Password

Total execution time: 00:00:00.005

In [40]:
EXEC WITHDRAW 1, 50000, 'P@SS'

: Msg 50000, Level 15, State 3, Procedure WITHDRAW, Line 47
Insufficient Balance

Total execution time: 00:00:00.013

# TRIGGERS

- Triggers are special functions that are automatically executed on certain DML/DDL events
- They can be executed
    - AFTER
        - INSERT
        - UPDATE
        - DELETE
        - SELECT
        - CREATE
        - ALTER
        - DROP
    - INSTEAD OF
        - INSERT
        - UPDATE
        - ...

- We can perform additional tasks when these events occurs
    - Remember! They are called automatically

  

## CREATE A TRIGGER TO LOG ALL THE TRANSACTION TIME AND DETAIL IN A SEPARATE TABLE

In [42]:
CREATE TABLE TRANSACTION_LOG(
    ID INT IDENTITY CONSTRAINT PK_TRANSACTION_LOG_ID PRIMARY KEY,
    DATE DATETIME,
    TRANSACTION_TYPE INT,
    ACCOUNT_ID INT 
        CONSTRAINT FK__TRANSACTION_LOG__ACCOUNT_ID__BANK_ACCOUNT_ID 
        FOREIGN KEY REFERENCES BANKACCOUNTS(ID),
    DESCRIPTION VARCHAR(100),
);

Commands completed successfully.

Total execution time: 00:00:00.016

In [43]:
CREATE TRIGGER LogTransaction
ON TransactionInfo
AFTER INSERT
AS
BEGIN

    INSERT 
        INTO TRANSACTION_LOG(ACCOUNT_ID,TRANSACTION_TYPE, DESCRIPTION, DATE)
        -- GET IT FROM ANOTHER LOCATION
        SELECT
            ACCOUNT_ID,
            TRANSACTION_TYPE,
            DESCRIPTION,
            GETDATE()
        FROM
            INSERTED --NEWLY INSERTED VALUES ARE PROVIDED AS PSUDO TABLE INSERTED

END


Commands completed successfully.

Total execution time: 00:00:00.012

In [44]:
EXEC Deposit 1, 20000
EXEC Withdraw 1, 15000, 'P@SS'

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.039

In [45]:
SELECT * FROM TRANSACTION_LOG

(2 rows affected)

Total execution time: 00:00:00.003

ID,DATE,TRANSACTION_TYPE,ACCOUNT_ID,DESCRIPTION
1,2024-07-26 17:18:21.070,1,1,Deposit
2,2024-07-26 17:18:21.080,-1,1,Withdraw


# TRANSACTION

- A Transaction is a series of task that should either
    - run completely
    - not at all.
- Consider a Scenario
    - We want to transfer fund from one account to another
    - It has few basic steps

1. Withraw money from source/sender account
    
2. insert transactioninfo
    
3. update bankaccounts
    
4. Deposit money in benificiary/target account
    
5. insert transactioninfo
    
6. updatebankaccounts
    
    - Consider what happens if
        - benficiary account is invalid
            - money is withdrawn from source
            - it reached no where
        - there is a exception/crash in any of the step

## How transaction works

- A transaction has three phases

1. BEGIN
2. COMMIT \<--- SUCCESS
3. ROLLBACK \<--- FAILS

- EITHER ALL TRANSACTIONS ARE COMMITED OR ALL IS ROLLED BACK

## DATBASE FOLLOWS **ACID PRINCIPLE**

- **A--\> ATOMIC**
    
    - Each step is an atomic step
    - transaction is made up of several atomic steps
- **C --\> CONSISTENCY**
    
    - Database must always maintain a consistent state
        - Unless transaction is over, it should not have invalid value at any poit of time
        - If we update source account but not beneficiary then state is not consistent
            - system is missing some money at this point
- **I ---\> ISOLATION**
    
    - Each step can be peformed in isolation
    - Any row will be part of a single transaction at a time
    - if one row is part of one transaction, other must wait for it finish
- **D ---\> Durability**
    
    - Once a transaction is committed it becomes durable (peristent)
        
    - must be saved 
        
    - it should be available here onward
        
    

# Advantage of Transaction

- We don't maintain state manually
    
- we can insert/update/delete our records with transaction like normal
    
- but those transactions will not really make changes to the database
    
    - they will be stored virtually
    - until transaction is 
        - committed
            
            - all previous operations are persisted
            
        - rolledback
            - database was never changed in the first place

## TRANSFER WITHOUT TRANSACTION

In [55]:
CREATE PROCEDURE TRANSFER
@SOURCE INT,
@AMOUNT FLOAT,
@PASSWORD VARCHAR(100),
@TARGET INT
AS
BEGIN

    EXEC Withdraw @SOURCE, @AMOUNT, @PASSWORD;
    EXEC Deposit @TARGET, @AMOUNT
END

Commands completed successfully.

Total execution time: 00:00:00.012

In [51]:
DROP PROCEDURE SHOW_TABLES;

Commands completed successfully.

Total execution time: 00:00:00.028

In [52]:
CREATE PROCEDURE SHOW_TABLES
AS
BEGIN
    SELECT * FROM BANKACCOUNTS;
    SELECT * FROM TRANSACTIONINFO;
    SELECT * FROM TRANSACTION_LOG;
END

Commands completed successfully.

Total execution time: 00:00:00.026

In [53]:
EXEC SHOW_TABLES

(2 rows affected)

(4 rows affected)

(2 rows affected)

Total execution time: 00:00:00.017

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,10000
2,Sanjay,P@SS,5000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000
2,1,Withdraw,-1,1000
3,1,Deposit,1,20000
4,1,Withdraw,-1,15000


ID,DATE,TRANSACTION_TYPE,ACCOUNT_ID,DESCRIPTION
1,2024-07-26 17:18:21.070,1,1,Deposit
2,2024-07-26 17:18:21.080,-1,1,Withdraw


### IT WILL WORK IN VALID CASE

In [56]:
EXEC TRANSFER 1, 2000, 'P@SS',2

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.029

In [57]:
EXEC SHOW_TABLES

(2 rows affected)

(6 rows affected)

(4 rows affected)

Total execution time: 00:00:00.023

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,8000
2,Sanjay,P@SS,7000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000
2,1,Withdraw,-1,1000
3,1,Deposit,1,20000
4,1,Withdraw,-1,15000
5,1,Withdraw,-1,2000
6,2,Deposit,1,2000


ID,DATE,TRANSACTION_TYPE,ACCOUNT_ID,DESCRIPTION
1,2024-07-26 17:18:21.070,1,1,Deposit
2,2024-07-26 17:18:21.080,-1,1,Withdraw
3,2024-07-26 17:38:21.657,-1,1,Withdraw
4,2024-07-26 17:38:21.670,1,2,Deposit


BUT DATABASE WILL BECOME INCONSITENT IF BENIFICARY ACCOUNT IS WORNG

In [58]:
EXEC TRANSFER 1, 2000, 'P@SS',100

(1 row affected)

(1 row affected)

(1 row affected)

: Msg 50000, Level 15, State 1, Procedure Deposit, Line 22
Invalid Account Number

Total execution time: 00:00:00.033

In [60]:
EXEC SHOW_TABLES

(2 rows affected)

(7 rows affected)

(5 rows affected)

Total execution time: 00:00:00.009

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,6000
2,Sanjay,P@SS,7000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000
2,1,Withdraw,-1,1000
3,1,Deposit,1,20000
4,1,Withdraw,-1,15000
5,1,Withdraw,-1,2000
6,2,Deposit,1,2000
7,1,Withdraw,-1,2000


ID,DATE,TRANSACTION_TYPE,ACCOUNT_ID,DESCRIPTION
1,2024-07-26 17:18:21.070,1,1,Deposit
2,2024-07-26 17:18:21.080,-1,1,Withdraw
3,2024-07-26 17:38:21.657,-1,1,Withdraw
4,2024-07-26 17:38:21.670,1,2,Deposit
5,2024-07-26 17:39:56.270,-1,1,Withdraw


# APPLYING TRANSACTION

In [61]:
ALTER PROCEDURE TRANSFER
@SOURCE INT,
@AMOUNT FLOAT,
@PASSWORD VARCHAR(100),
@TARGET INT
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        EXEC Withdraw @SOURCE, @AMOUNT, @PASSWORD;
        EXEC Deposit @TARGET, @AMOUNT
        COMMIT TRANSACTION
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        RAISERROR('Trnasaction Failed',15,4)
    END CATCH
END

Commands completed successfully.

Total execution time: 00:00:00.012

In [62]:
EXEC Transfer 1, 1000, 'P@SS',2

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.033

In [63]:
EXEC SHOW_TABLES

(2 rows affected)

(9 rows affected)

(7 rows affected)

Total execution time: 00:00:00.009

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,5000
2,Sanjay,P@SS,8000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000
2,1,Withdraw,-1,1000
3,1,Deposit,1,20000
4,1,Withdraw,-1,15000
5,1,Withdraw,-1,2000
6,2,Deposit,1,2000
7,1,Withdraw,-1,2000
8,1,Withdraw,-1,1000
9,2,Deposit,1,1000


ID,DATE,TRANSACTION_TYPE,ACCOUNT_ID,DESCRIPTION
1,2024-07-26 17:18:21.070,1,1,Deposit
2,2024-07-26 17:18:21.080,-1,1,Withdraw
3,2024-07-26 17:38:21.657,-1,1,Withdraw
4,2024-07-26 17:38:21.670,1,2,Deposit
5,2024-07-26 17:39:56.270,-1,1,Withdraw
6,2024-07-26 17:46:38.870,-1,1,Withdraw
7,2024-07-26 17:46:38.877,1,2,Deposit


In [64]:
EXEC TRANSFER 2, 5000, 'P@SS', 4

(1 row affected)

(1 row affected)

(1 row affected)

: Msg 50000, Level 15, State 4, Procedure TRANSFER, Line 17
Trnasaction Failed

Total execution time: 00:00:00.029

In [65]:
EXEC SHOW_TABLES

(2 rows affected)

(9 rows affected)

(7 rows affected)

Total execution time: 00:00:00.008

ID,NAME,PASSWORD,AMOUNT
1,Vivek,P@SS,5000
2,Sanjay,P@SS,8000


ID,ACCOUNT_ID,DESCRIPTION,TRANSACTION_TYPE,AMOUNT
1,1,Deposit,1,1000
2,1,Withdraw,-1,1000
3,1,Deposit,1,20000
4,1,Withdraw,-1,15000
5,1,Withdraw,-1,2000
6,2,Deposit,1,2000
7,1,Withdraw,-1,2000
8,1,Withdraw,-1,1000
9,2,Deposit,1,1000


ID,DATE,TRANSACTION_TYPE,ACCOUNT_ID,DESCRIPTION
1,2024-07-26 17:18:21.070,1,1,Deposit
2,2024-07-26 17:18:21.080,-1,1,Withdraw
3,2024-07-26 17:38:21.657,-1,1,Withdraw
4,2024-07-26 17:38:21.670,1,2,Deposit
5,2024-07-26 17:39:56.270,-1,1,Withdraw
6,2024-07-26 17:46:38.870,-1,1,Withdraw
7,2024-07-26 17:46:38.877,1,2,Deposit
